---
# 
---

In this notebook, 

1. 


### Importing necessary libraries

In [68]:
import os
import glob

import pandas as pd
import numpy as np

from geopy import distance

from id_station import stn_id 

### Define files

In [7]:
stn_inventory = '../data/ECCC/Station_Inventory_EN_AdminRegion.csv'

### Define variables

In [48]:
admin_region = ['Montréal']

data_dir = '../data/ECCC/daily'

filename = 'climate_daily_QC_'

frequency = 'DLY'

### Read Station Inventory file

In [9]:
stns = pd.read_csv(stn_inventory)
stns.head()

,Name,Province,Climate ID,Station ID,WMO ID,TC ID,Latitude (Decimal Degrees),Longitude (Decimal Degrees),Latitude,Longitude,...,First Year,Last Year,HLY First Year,HLY Last Year,DLY First Year,DLY Last Year,MLY First Year,MLY Last Year,geometry,admin
0,BARRIERE STONEHAM,QUEBEC,7010478,5205,NaN,NaN,47.17,-71.25,471000000,-711500000,...,1963,1976,NaN,NaN,1963.0,1976.0,1963.0,1976.0,POINT (-71.25 47.17),Capitale-Nationale
1,BARRIERE TOURILLI,QUEBEC,7010480,5206,NaN,NaN,47.17,-71.62,471000000,-713700000,...,1949,1960,NaN,NaN,1949.0,1960.0,1949.0,1960.0,POINT (-71.62 47.17),Capitale-Nationale
2,BEAUPORT,QUEBEC,7010565,27803,71578.0,XBO,46.84,-71.20,465013000,-711150000,...,1999,2025,1999.0,2024.0,1999.0,2025.0,NaN,NaN,POINT (-71.2 46.84),Capitale-Nationale
3,BEAUPORT,QUEBEC,7010566,5207,NaN,NaN,46.88,-71.20,465300000,-711200000,...,1982,1985,NaN,NaN,1982.0,1985.0,1982.0,1985.0,POINT (-71.2 46.88),Capitale-Nationale
4,BERTHIERVILLE,QUEBEC,7010720,5208,NaN,NaN,46.05,-73.18,460300000,-731100000,...,1919,1995,NaN,NaN,1919.0,1995.0,1919.0,1995.0,POINT (-73.18 46.05),Lanaudière


### Keep only stations within the desired administravtive region

In [14]:
stns = stns[stns['admin'].isin(admin_region)]
stns

,Name,Province,Climate ID,Station ID,WMO ID,TC ID,Latitude (Decimal Degrees),Longitude (Decimal Degrees),Latitude,Longitude,...,First Year,Last Year,HLY First Year,HLY Last Year,DLY First Year,DLY Last Year,MLY First Year,MLY Last Year,geometry,admin
160,COTE ST LUC,QUEBEC,7021945,5343,NaN,NaN,45.45,-73.67,452700000,-734000000,...,1960,1963,NaN,NaN,1960.0,1963.0,1960.0,1963.0,POINT (-73.67 45.45),Montréal
217,LA SALLE,QUEBEC,7024118,5391,NaN,NaN,45.43,-73.62,452600000,-733700000,...,1973,1983,NaN,NaN,1973.0,1983.0,1973.0,1983.0,POINT (-73.62 45.43),Montréal
220,LAVAL DES RAPIDES,QUEBEC,7024256,5394,NaN,NaN,45.53,-73.70,453200000,-734200000,...,1965,1976,NaN,NaN,1965.0,1976.0,1965.0,1976.0,POINT (-73.7 45.53),Montréal
226,MACDONALD COLLEGE,QUEBEC,7024400,5400,NaN,NaN,45.42,-73.93,452500000,-735600000,...,1906,1976,NaN,NaN,1906.0,1976.0,1906.0,1976.0,POINT (-73.93 45.42),Montréal
234,MCTAVISH,QUEBEC,7024745,10761,71612.0,WTA,45.50,-73.58,453017070,-733445000,...,1994,2025,1994.0,2024.0,1994.0,2025.0,NaN,NaN,POINT (-73.58 45.5),Montréal
239,MOBILE UPPER AIR STATION-QUEBEC,QUEBEC,7025000,47888,71054.0,EUQ,45.43,-73.93,452536000,-735544040,...,2014,2021,2014.0,2021.0,2017.0,2021.0,NaN,NaN,POINT (-73.93 45.43),Montréal
242,MONTREAL ADAC A,QUEBEC,7025228,8343,NaN,NaN,45.48,-73.55,452900000,-733300000,...,1974,1976,1974.0,1976.0,NaN,NaN,NaN,NaN,POINT (-73.55 45.48),Montréal
245,MONTREAL/PIERRE ELLIOTT TRUDEAU INTL A,QUEBEC,7025250,5415,71627.0,YUL,45.47,-73.75,452800000,-734500000,...,1941,2013,1953.0,2013.0,1941.0,2013.0,1941.0,2013.0,POINT (-73.75 45.47),Montréal
246,MONTREAL INTL A,QUEBEC,7025251,51157,71627.0,YUL,45.47,-73.74,452814000,-734427000,...,2013,2025,2013.0,2024.0,2013.0,2025.0,NaN,NaN,POINT (-73.74 45.47),Montréal
247,MONTREAL-EST,QUEBEC,7025252,26855,NaN,WPQ,45.63,-73.55,453805000,-733311000,...,1994,2008,1994.0,2008.0,1994.0,2008.0,NaN,NaN,POINT (-73.55 45.63),Montréal


### Creating a dictionary of the reference station to preprocess

In [42]:
tmp = []

for key in stn_id.keys():
    for id in stn_id[key]:
        ref_stn[key][id] = {}

        lat = stns[stns['Climate ID'] == str(id)]['Latitude (Decimal Degrees)'].values[0]
        lon = stns[stns['Climate ID'] == str(id)]['Longitude (Decimal Degrees)'].values[0]

        admin = stns[stns['Climate ID'] == str(id)]['admin'].values[0]

        tmp.append({'Station': key, 
                    'Climate ID': id, 
                    'Latitude': lat, 
                    'Longitude': lon, 
                    'admin': admin})
        
ref_stn = pd.DataFrame(tmp)
ref_stn

,Station,Climate ID,Latitude,Longitude,admin
0,Montreal (McTavish/McGill),7025280,45.5,-73.58,Montréal
1,Montreal (McTavish/McGill),7024745,45.5,-73.58,Montréal


### Compute the distance from the stations to the reference station

In [47]:
dist_stn_ref = []

for admin in admin_region :
    tmp_stns = stns[stns['admin'].isin([admin])]
    tmp_ref_stn = ref_stn[ref_stn['admin'].isin([admin])]

    for index, ref in tmp_ref_stn.iterrows():
        for idx, stn in tmp_stns.iterrows() :
            
            ref_lat_lon = (ref['Latitude'], ref['Longitude'])
            stn_lat_lon = (stn['Latitude (Decimal Degrees)'], stn['Longitude (Decimal Degrees)'])
            
            dist = distance.distance(ref_lat_lon, stn_lat_lon).km

            dist_stn_ref.append({'Station' : ref['Station'],
                                 'Climate ID' : ref['Climate ID'],
                                 'Latitude': ref['Latitude'],
                                 'Longitude':  ref['Longitude'],
                                 'admin': admin,
                                 'Nearby Climate ID': stn['Climate ID'],
                                 'Distance (km)': dist
            })
            
            
dist_stn_ref = pd.DataFrame(dist_stn_ref)
dist_stn_ref

,Station,Climate ID,Latitude,Longitude,admin,Nearby Climate ID,Distance (km)
0,Montreal (McTavish/McGill),7025280,45.5,-73.58,Montréal,7021945,8.966881
1,Montreal (McTavish/McGill),7025280,45.5,-73.58,Montréal,7024118,8.385239
2,Montreal (McTavish/McGill),7025280,45.5,-73.58,Montréal,7024256,9.951661
3,Montreal (McTavish/McGill),7025280,45.5,-73.58,Montréal,7024400,28.782395
4,Montreal (McTavish/McGill),7025280,45.5,-73.58,Montréal,7024745,0.000000
5,Montreal (McTavish/McGill),7025280,45.5,-73.58,Montréal,7025000,28.456381
6,Montreal (McTavish/McGill),7025280,45.5,-73.58,Montréal,7025228,3.231210
7,Montreal (McTavish/McGill),7025280,45.5,-73.58,Montréal,7025250,13.702253
8,Montreal (McTavish/McGill),7025280,45.5,-73.58,Montréal,7025251,12.945362
9,Montreal (McTavish/McGill),7025280,45.5,-73.58,Montréal,7025252,14.637152


### Reading the data files of reference station

In [78]:
for idx, ref in ref_stn.iterrows():
    start = int(stns[stns['Climate ID'] == str(ref['Climate ID'])][f'{frequency} First Year'].values[0])
    end = int(stns[stns['Climate ID'] == str(ref['Climate ID'])][f'{frequency} Last Year'].values[0])

    print(start, end)

    for year in list(range(start,end+1)):
        path = os.path.join(data_dir, f'*{ref['Climate ID']}_{year}*.csv')
        file = glob.glob(path)
        print(file[0])
        data = pd.read_csv(file[0], encoding="ISO-8859-1")

        print(data)
        break

1871 1993
../data/ECCC/daily/climate_daily_QC_7025280_1871_P1D.csv
     Longitude (x)  Latitude (y)     Station Name  Climate ID   Date/Time  \
0           -73.58          45.5  MONTREAL MCGILL     7025280  1871-01-01   
1           -73.58          45.5  MONTREAL MCGILL     7025280  1871-01-02   
2           -73.58          45.5  MONTREAL MCGILL     7025280  1871-01-03   
3           -73.58          45.5  MONTREAL MCGILL     7025280  1871-01-04   
4           -73.58          45.5  MONTREAL MCGILL     7025280  1871-01-05   
..             ...           ...              ...         ...         ...   
360         -73.58          45.5  MONTREAL MCGILL     7025280  1871-12-27   
361         -73.58          45.5  MONTREAL MCGILL     7025280  1871-12-28   
362         -73.58          45.5  MONTREAL MCGILL     7025280  1871-12-29   
363         -73.58          45.5  MONTREAL MCGILL     7025280  1871-12-30   
364         -73.58          45.5  MONTREAL MCGILL     7025280  1871-12-31   

     Yea